# Chat Bot

### Importing the lbraries

In [1]:
import numpy as np
import tensorflow as tf
import time 
import pandas as pd
import re


### DATA Preprocessing 

In [21]:
lines  = open("movie_lines.txt",encoding='utf-8',errors='ignore').read().split('\n')
conversations =  open("movie_conversations.txt",encoding='utf-8',errors='ignore').read().split('\n')

In [23]:
idline = {}
for line in lines:
    line_ = line.split(" +++$+++ ")
    if len(line_)==5:
        idline[line_[0]]=line_[4]

In [25]:
conversations_id = []
for conversation in conversations:
    conversation_ = conversation.split(" +++$+++ ")[-1][1:-1].replace("'","").replace(" ","")
    conversations_id.append(conversation_.split(","))

In [27]:
questions= []
awnsers = []
for convers in conversations_id:
    for i in range(len(convers)-1):
        questions.append(idline[convers[i]])
        awnsers.append(idline[convers[i+1]])
    

In [42]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"\'ll'"," will",text)
    text = re.sub(r"\'ve'"," have",text)
    text = re.sub(r"\'re'"," are",text)
    text = re.sub(r"\'d'"," would",text)
    text = re.sub(r"didn't","did not",text)
    text = re.sub(r"we'd","we would",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"won't","will not",text)
    text = re.sub(r"can't","can not",text)
    text = re.sub(r"[-(){}\"#$%^&*!@:;+=.|,]","",text)
    return text
    

In [80]:
clean_questions = []
for quest in questions:
    clean_questions.append(clean_text(quest))
clean_awnsers= []
for ans in awnsers:
    clean_awnsers.append(clean_text(ans))

In [81]:
word2count = {}
for word in clean_questions:
    for w in  word.split():
        if i not in word2count:
            word2count[i]=1
        else: word2count[i]+=1
for word in clean_awnsers:
    for w in  word.split():
        if i not in word2count:
            word2count[i]=1
        else: word2count[i]+=1


In [82]:
threshold =20
questionswords2int ={}
wordnumber = 0

for word,count in word2count.items():
    if count>=threshold:
        questionswords2int[word] = wordnumber
        wordnumber+=1

anwserswords2int ={}
wordnumber = 0

for word,count in word2count.items():
    if count >= threshold:
        anwserswords2int[word] = wordnumber
        wordnumber+=1

        

In [83]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    anwserswords2int[token] = len(questionswords2int) + 1
    


In [84]:
anwsersints2words = {w_i: w for w ,w_i in anwserswords2int.items() }

In [85]:
for i in range(len(clean_awnsers)):
    clean_awnsers[i] += "<EOS>"

In [86]:
questions_to_int = []
for question in  clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_to_int.append(ints)

anwsers_to_int = []
for answsers in  clean_awnsers:
    ints = []
    for word in answsers.split():
        if word not in anwserswords2int:
            ints.append(anwserswords2int['<OUT>'])
        else:
            ints.append(anwserswords2int[word])
    anwsers_to_int.append(ints)
            

In [88]:
sorted_clean_question = []
sorted_clean_anwsers = []
for length in range(1,25+1):
    for i in enumerate(questions_to_int):
        if len(i[1])==length:
            sorted_clean_question.append(questions_to_int[i[0]])
            sorted_clean_question.append(anwsers_to_int[i[0]])

### Building seq-seq model

In [89]:
def model_input():
    inputs = tf.placeholder(tf.int32,[None,None],name='input')
    targets = tf.placeholder(tf.int32,[None,None],name='target')
    lr = tf.placeholder(tf.float32,name='learning_rate')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    
    return inputs,targets,lr,keep_prob

In [90]:
def preprocess_targets(targets,words2int,batch_size):
    left_side = tf.fill([batch_size, 1],word2int['<SOS>'])
    right_side = tf.strided_slice(targets,[0,0],[batch_size,-1],[1,1])
    preprocessed_targets = tf.concat([left_side,right_side],1)
    return preprocessed_targets
    

In [91]:
def encoder_rnn_layer(rnn_inputs,rnn_size,num_layers,keep_prob,sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout]*num_layers)
    _,encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw  = encoder_cell,cell_bw=encoder_cell,sequence_length=sequence_length,inputs=rnn_inputs,dtype=tf.float32)  
    return encoder_state